In [1]:
!pip install --upgrade streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00


In [2]:
import streamlit as st
print("Streamlit version:", st.__version__)

Streamlit version: 1.35.0


In [3]:
!pip show streamlit #shwos latest version of the streamlit


Name: streamlit
Version: 1.35.0
Summary: A faster way to build and share data apps
Home-page: https://streamlit.io
Author: Snowflake Inc
Author-email: hello@streamlit.io
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: altair, blinker, cachetools, click, gitpython, numpy, packaging, pandas, pillow, protobuf, pyarrow, pydeck, requests, rich, tenacity, toml, tornado, typing-extensions, watchdog
Required-by: 


In [4]:
!pip install urlextract

In [5]:
import pandas as pd
import re
import matplotlib.pyplot as plt

In [6]:
!pip install wordcloud

In [7]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 2.8 MB/s eta 0:00:00


In [16]:
%%writefile app.py
import streamlit as st
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
#->preprocessor.py starts
def preprocess(data):
  pattern='\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'
  messages = re.split(pattern,data)[1:]
  dates=re.findall(pattern,data)
  # Create DataFrame
  df = pd.DataFrame({'user_message': messages, 'message_date': dates})
  # Convert message_date type
  df['message_date'] = pd.to_datetime(df['message_date'], format='%m/%d/%y, %H:%M - ')
  df.rename(columns={'message_date': 'date'}, inplace=True)
  #seperating the user name and user messages from single column
  users=[]
  messages=[]
  for message in df['user_message']:
    entry=re.split('([\w\W]+?):\s',message)
    if entry[1:]:
      users.append(entry[1])
      messages.append(entry[2])
    else:
      users.append('group_notification')
      messages.append(entry[0])
  df['user']=users
  df['message']=messages
  df.drop(columns=['user_message'],inplace=True)
  df['year']=df['date'].dt.year
  df['month']=df['date'].dt.month_name()
  df['month_number']=df['date'].dt.month #for showing monthly timeline
  df['only_date']=df['date'].dt.date #used in daily timeline
  df['day_name']=df['date'].dt.day_name() #Used in daily activity map
  df['day']=df['date'].dt.day
  df['hour']=df['date'].dt.hour
  df['minute']=df['date'].dt.minute
  #for Heatmap
  period =[]
  for hour in df[['day_name','hour']]['hour']:
    if hour==23:
      period.append(str(hour) + "-" + ('00'))
    elif hour==0:
      period.append(str('00')+ "-"+ str(hour+1))
    else:
      period.append(str(hour)+ "-"+ str(hour+1))
  df['period']=period #For Heatmap
  return df
#preprocssor.py ends <-
#-> Helper.py stars
def fetch_stats(selected_user,df):
  if (selected_user != "Overall") :
    df=df[df['user'] == selected_user]
  #1.fetching total messages
  #num_messages = df.shape[0]
  num_messages=df['message'].count()
  #2. fetching total words
  words=[]
  for message in df['message']:
    words.extend(message.split())
  total_words = len(words)
  #3.fetching total media messages
  num_media_msg = df[df['message'] =='<Media omitted>\n'].shape[0]
  #4.feteching total links
  from urlextract import URLExtract
  extractor = URLExtract()
  y = []
  for message in df['message']:
    y.extend(extractor.find_urls(message))
  num_links=len(y)
  return num_messages,total_words,num_media_msg,num_links
#Most busy user
def most_act_user(df):
  x=df['user'].value_counts().head()
  new_df=round((df['user'].value_counts().head(33)/df.shape[0])*100,2).reset_index().rename(columns={'user':'Name','count':'Percentage'})
  return x,new_df
#Word cloud
from wordcloud import WordCloud
def word_cloud(selected_user,df):
  f = open("/content/drive/MyDrive/stop_hinglish.txt",'r')
  stop_words = f.read()
  if selected_user!='Overall':
    df=df[df["user"]== selected_user]
  temp=df[df['user']!='group_notification']
  temp=temp[temp['message']!='<Media omitted>\n']
  temp['message'] = temp['message'].apply(lambda x: re.sub(r'\d+', '', x))
  temp['message'] = temp['message'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
  def remove_stop_words(message):
    words=[]
    for message in temp['message']:
      for word in message.lower().split():
        if(word not in stop_words):
          words.append(word)
    return " ".join(words)
  wc= WordCloud(width=500,height=500,min_font_size=10,background_color="black")
  temp['message'] = temp['message'].apply(remove_stop_words)
  df_wc = wc.generate(temp['message'].str.cat(sep=" "))
  return df_wc
#Most word used
def most_word_used(selected_user,df):
  f = open("/content/drive/MyDrive/stop_hinglish.txt",'r')
  stop_words = f.read()
  if selected_user!='Overall':
    df=df[df["user"]== selected_user]
  temp=df[df['user']!='group_notification']
  temp=temp[temp['message']!='<Media omitted>\n']
  temp['message'] = temp['message'].apply(lambda x: re.sub(r'\d+', '', x))
  temp['message'] = temp['message'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
  words=[]
  for message in temp['message']:
    for word in message.lower().split():
      if(word not in stop_words):
        words.append(word)
  from collections import Counter
  most_common_df=pd.DataFrame(Counter(words).most_common(20))
  return most_common_df
#Emojies
import emoji
def emojis_used(selected_user,df):
  from collections import Counter
  if selected_user!='Overall':
    df=df[df["user"]== selected_user]
  emojis=[]
  for message in df['message']:
    emojis.extend([c for c in message if c in emoji.distinct_emoji_list(message)])
  emoji_df = pd.DataFrame(Counter(emojis).most_common(len(Counter(emojis)))).head(100)
  return emoji_df
#Monthly timeline
def show_monthly_timeline(selected_user,df):
  if selected_user!='Overall':
    df=df[df["user"]== selected_user]
  timeline=df.groupby(['year','month_number','month']).count()['message'].reset_index()
  time=[]
  for i in range(timeline.shape[0]):
    time.append(timeline['month'][i] + "-" + str(timeline['year'][i]))
  timeline['time']=time
  return timeline
#Daily timeline
def show_daily_timeline(selected_user,df):
  if selected_user!='Overall':
    df=df[df["user"]== selected_user]
  daily_timeline=df.groupby('only_date').count()['message'].reset_index()
  return daily_timeline
#Daily Activity-Map
def show_week_activity_map(selected_user,df):
  if selected_user!='Overall':
    df=df[df["user"]== selected_user]
  return df['day_name'].value_counts()
def show_month_activity_map(selected_user,df):
  if selected_user!='Overall':
    df=df[df["user"]== selected_user]
  return df['month'].value_counts()
#Activity Heatmap
def activity_heatmap(selected_user,df):
  if selected_user!='Overall':
    df=df[df["user"]== selected_user]
  user_heatmap=df.pivot_table(index='day_name',columns='period',values='message',aggfunc='count').fillna(0)
  return user_heatmap
  #ends <-

#->Streamlit code Starts
#st.write("Whatsapp chat analyzer")
st.sidebar.title("Whatsapp chat analyzer")
uploaded_file = st.sidebar.file_uploader("Choose a file")
if uploaded_file is not None:
    bytes_data = uploaded_file.getvalue()
    data = bytes_data.decode("utf-8")
    #st.text(data)
    df=preprocess(data)
    #st.dataframe(df)
    user_list=df['user'].unique().tolist()
    user_list.remove('group_notification')
    user_list.sort()
    user_list.insert(0,'Overall')
    selected_user=st.sidebar.selectbox("show analysis wrt",user_list)
    if st.sidebar.button("Show analysis"):
      num_messages,total_words,num_media_msg,num_links = fetch_stats(selected_user,df)
      col1,col2,col3,col4 = st.columns(4)
      with col1:
        st.header("Total Messages")
        st.title(num_messages)
      with col2:
        st.header("Total Words")
        st.title(total_words)
      with col3:
        st.header("Total media messages")
        st.title(num_media_msg)
      with col4:
        st.header("Total links")
        st.title(num_links)
      #Monthly timeline
      st.title("Monthly timeline")
      timeline=show_monthly_timeline(selected_user,df)
      fig,ax=plt.subplots()
      ax.plot(timeline['time'],timeline['message'],color='blue')
      plt.xticks(rotation='vertical')
      st.pyplot(fig)
      #Daily Timeline
      st.title("Daily Timeline")
      daily_timeline=show_daily_timeline(selected_user,df)
      plt.figure(figsize=(20,20))
      fig,ax=plt.subplots()
      ax.plot(daily_timeline['only_date'],daily_timeline['message'],color='blue')
      plt.xticks(rotation='vertical')
      st.pyplot(fig)
      #Daily-Activity map
      st.title("Activity map")
      col1,col2=st.columns(2)
      with col1:
        st.header("Most busy day")
        busy_day=show_week_activity_map(selected_user,df)
        fig,ax=plt.subplots()
        ax.bar(busy_day.index,busy_day.values,color='blue')
        plt.xticks(rotation='vertical')
        st.pyplot(fig)
      with col2:
        st.header("Most busy month")
        busy_month=show_month_activity_map(selected_user,df)
        fig,ax=plt.subplots()
        ax.bar(busy_month.index,busy_month.values,color='orange')
        plt.xticks(rotation='vertical')
        st.pyplot(fig)
      #Activity Heat Map
      st.title("Weekly Activity Heatmap")
      user_heatmap=activity_heatmap(selected_user,df)
      fig,ax=plt.subplots()
      ax=sns.heatmap(user_heatmap)
      st.pyplot(fig)
      #finding most active person
      if selected_user == 'Overall':
        x ,new_df= most_act_user(df)
        fig, ax = plt.subplots()
        col1,col2=st.columns(2)
        with col1:
          st.title("Most busy users")
          ax.bar(x.index, x.values,color='green')
          plt.xticks(rotation ='vertical')
          st.pyplot(fig)
        with col2:
          st.dataframe(new_df)
  #wordcloud
      st.title("WordCloud")
      df_wc = word_cloud(selected_user,df)
      fig,ax = plt.subplots()
      ax.imshow(df_wc)
      st.pyplot(fig)
  #Most word used
      most_common_df = most_word_used(selected_user,df)
      #col1,col2=st.columns(2)
      #with col1:
      st.title("Most Common Words")
      st.dataframe(most_common_df)
      #with col2:
      fig,ax=plt.subplots()
      ax.barh(most_common_df[0],most_common_df[1])
      plt.xticks(rotation='vertical')
      st.pyplot(fig)
  #Emojis analysis
      emoji_df=emojis_used(selected_user,df)
      col1,col2=st.columns(2)
      with col1:
        st.title("Emojis analysis")
        st.dataframe(emoji_df)
      with col2:
        fig,ax = plt.subplots()
        ax.pie(emoji_df[1].head(),labels=emoji_df[0].head(),autopct='%0.2f')
        st.pyplot(fig)
# Streamlit ends <-



Overwriting app.py


In [9]:
!pip install pyngrok

In [10]:
from pyngrok import ngrok

In [11]:
ngrok.set_auth_token("2gNAUjeCb7jy9O4y10gQPF3TBWs_6Uo2eLco8jBzT1tjJxeaK")

In [12]:
!nohup streamlit run app.py --server.port 5500 &

# Import necessary library
#from pyngrok import ngrok

# Establish the ngrok tunnel
ngrok_tunnel = ngrok.connect(addr='5500', proto='http')
print("Ngrok Tunnel URL:", ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
Ngrok Tunnel URL: https://606c-34-122-95-161.ngrok-free.app


In [13]:
'''from pyngrok import ngrok
tunnels = ngrok.get_tunnels()
tunnels'''
#ngrok.kill()#is used to delete tunnel

'from pyngrok import ngrok\ntunnels = ngrok.get_tunnels()\ntunnels'